In [1]:
# 1. Define our items (movies) and their features (genres)
movies = {
    "Movie A": {"genre": ["Action", "Sci-Fi"]},
    "Movie B": {"genre": ["Comedy", "Romance"]},
    "Movie C": {"genre": ["Action", "Adventure"]},
    "Movie D": {"genre": ["Sci-Fi", "Thriller"]},
    "Movie E": {"genre": ["Comedy", "Family"]},
    "Movie F": {"genre": ["Action", "Thriller"]},
    "Movie G": {"genre": ["Drama", "Romance"]},
    "Movie H": {"genre": ["Sci-Fi", "Adventure"]},
}

# 2. Define users and their preferences (movies they've liked)
users_preferences = {
    "Alice": ["Movie A", "Movie D"],
    "Bob": ["Movie B", "Movie E"],
    "Charlie": ["Movie A", "Movie C", "Movie F"],
    "Diana": ["Movie G"]
}

def get_liked_genres(user_name, user_prefs, all_movies):
    """Gets a set of all genres the user has liked."""
    liked_genres = set()
    if user_name in user_prefs:
        for movie_title in user_prefs[user_name]:
            if movie_title in all_movies:
                for genre in all_movies[movie_title]["genre"]:
                    liked_genres.add(genre)
    return liked_genres

def recommend_movies_content_based(user_name, user_prefs, all_movies, num_recommendations=3):
    """Recommends movies to a user based on content (genre similarity)."""
    recommendations = []
    
    if user_name not in user_prefs:
        print(f"Sorry, we don't have preferences for {user_name}.")
        return recommendations

    liked_genres = get_liked_genres(user_name, user_prefs, all_movies)
    
    if not liked_genres:
        print(f"Could not determine liked genres for {user_name}.")
        return recommendations

    print(f"\n--- Recommendations for {user_name} ---")
    print(f"Based on their liking for genres: {', '.join(liked_genres)}")

    # Create a dictionary to score movies based on genre matches
    movie_scores = {}

    for movie_title, movie_data in all_movies.items():
        # Don't recommend movies the user has already liked
        if movie_title in user_prefs[user_name]:
            continue

        score = 0
        for genre in movie_data["genre"]:
            if genre in liked_genres:
                score += 1  # Increment score for each matching genre
        
        if score > 0:
            movie_scores[movie_title] = score
            
    # Sort movies by score in descending order
    sorted_recommendations = sorted(movie_scores.items(), key=lambda item: item[1], reverse=True)
    
    # Get the top N recommendations
    for movie_title, score in sorted_recommendations:
        if len(recommendations) < num_recommendations:
            recommendations.append(movie_title)
        else:
            break
            
    return recommendations

# --- Let's get some recommendations! ---

# Recommendations for Alice
alice_liked_genres = get_liked_genres("Alice", users_preferences, movies)
print(f"Alice has liked movies with genres: {alice_liked_genres}")
alice_recs = recommend_movies_content_based("Alice", users_preferences, movies, num_recommendations=2)
if alice_recs:
    print(f"Recommended for Alice: {', '.join(alice_recs)}")

print("-" * 30)

# Recommendations for Bob
bob_liked_genres = get_liked_genres("Bob", users_preferences, movies)
print(f"Bob has liked movies with genres: {bob_liked_genres}")
bob_recs = recommend_movies_content_based("Bob", users_preferences, movies)
if bob_recs:
    print(f"Recommended for Bob: {', '.join(bob_recs)}")

print("-" * 30)

# Recommendations for Charlie
charlie_recs = recommend_movies_content_based("Charlie", users_preferences, movies)
if charlie_recs:
    print(f"Recommended for Charlie: {', '.join(charlie_recs)}")

print("-" * 30)

# Recommendations for a new user (or user with no genre overlap)
david_recs = recommend_movies_content_based("David", users_preferences, movies)
if not david_recs:
    print("No recommendations for David based on current data.")

print("-" * 30)

# Recommendations for Diana (likes Drama, Romance)
diana_recs = recommend_movies_content_based("Diana", users_preferences, movies)
if diana_recs:
    print(f"Recommended for Diana: {', '.join(diana_recs)}")

Alice has liked movies with genres: {'Action', 'Thriller', 'Sci-Fi'}

--- Recommendations for Alice ---
Based on their liking for genres: Action, Thriller, Sci-Fi
Recommended for Alice: Movie F, Movie C
------------------------------
Bob has liked movies with genres: {'Family', 'Romance', 'Comedy'}

--- Recommendations for Bob ---
Based on their liking for genres: Family, Romance, Comedy
Recommended for Bob: Movie G
------------------------------

--- Recommendations for Charlie ---
Based on their liking for genres: Action, Thriller, Sci-Fi, Adventure
Recommended for Charlie: Movie D, Movie H
------------------------------
Sorry, we don't have preferences for David.
No recommendations for David based on current data.
------------------------------

--- Recommendations for Diana ---
Based on their liking for genres: Romance, Drama
Recommended for Diana: Movie B


# Personal Gemini advanced acct

In [1]:
def simple_content_based_recommender(user_liked_items, all_items_features, num_recommendations=3):
    """
    A very simple content-based recommender system.

    Args:
        user_liked_items (list): A list of item IDs that the user has liked.
        all_items_features (dict): A dictionary where keys are item IDs and
                                   values are sets of features for that item.
        num_recommendations (int): The number of recommendations to return.

    Returns:
        list: A list of recommended item IDs, sorted by similarity score.
    """

    # 1. Build User Profile: Aggregate features from items the user liked.
    user_profile_features = set()
    for item_id in user_liked_items:
        if item_id in all_items_features:
            user_profile_features.update(all_items_features[item_id])

    if not user_profile_features:
        print("User profile is empty. Cannot make recommendations based on current liked items.")
        return []

    print(f"\nUser Profile Features (based on liked items): {user_profile_features}")

    # 2. Calculate Similarity and Score Items:
    #    Find items the user hasn't seen and calculate their similarity
    #    to the user's profile using Jaccard similarity.
    recommendations = {}
    for item_id, features in all_items_features.items():
        if item_id not in user_liked_items:  # Only recommend new items
            intersection_size = len(user_profile_features.intersection(features))
            union_size = len(user_profile_features.union(features))

            if union_size == 0: # Avoid division by zero if both sets are empty (though unlikely here)
                similarity_score = 0
            else:
                similarity_score = intersection_size / union_size

            if similarity_score > 0: # Only consider items with some similarity
                recommendations[item_id] = similarity_score

    # 3. Sort and Return Top Recommendations:
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

    # Return only the item IDs
    return [item_id for item_id, score in sorted_recommendations[:num_recommendations]]

# --- Example Usage ---

# Define our items and their features (e.g., movies and their genres)
# Using sets for features makes intersection/union easier.
items_database = {
    'MovieA': {'Action', 'SciFi', 'Adventure'},
    'MovieB': {'Comedy', 'Romance'},
    'MovieC': {'Action', 'Thriller', 'Crime'},
    'MovieD': {'SciFi', 'Mystery', 'Thriller'},
    'MovieE': {'Romance', 'Drama'},
    'MovieF': {'Action', 'Comedy', 'Crime'},
    'MovieG': {'SciFi', 'Adventure', 'Family'},
    'MovieH': {'Documentary', 'History'},
    'MovieI': {'SciFi', 'Action', 'Thriller'} # Similar to A, C, D
}

# Simulate a user and the items they liked
current_user_likes = ['MovieA', 'MovieD']
print(f"User has liked: {current_user_likes}")

# Get recommendations for the user
recommended_items = simple_content_based_recommender(current_user_likes, items_database, num_recommendations=3)

if recommended_items:
    print(f"\nTop recommendations for the user:")
    for i, item_id in enumerate(recommended_items):
        print(f"{i+1}. {item_id} (Features: {items_database[item_id]})")
else:
    print("No recommendations found with the current criteria.")

print("\n--- Another Example: User with different tastes ---")
current_user_likes_2 = ['MovieB', 'MovieE']
print(f"User 2 has liked: {current_user_likes_2}")
recommended_items_2 = simple_content_based_recommender(current_user_likes_2, items_database, num_recommendations=2)
if recommended_items_2:
    print(f"\nTop recommendations for User 2:")
    for i, item_id in enumerate(recommended_items_2):
        print(f"{i+1}. {item_id} (Features: {items_database[item_id]})")
else:
    print("No recommendations found for User 2 with the current criteria.")

User has liked: ['MovieA', 'MovieD']

User Profile Features (based on liked items): {'SciFi', 'Thriller', 'Mystery', 'Action', 'Adventure'}

Top recommendations for the user:
1. MovieI (Features: {'Action', 'Thriller', 'SciFi'})
2. MovieC (Features: {'Action', 'Thriller', 'Crime'})
3. MovieG (Features: {'Family', 'SciFi', 'Adventure'})

--- Another Example: User with different tastes ---
User 2 has liked: ['MovieB', 'MovieE']

User Profile Features (based on liked items): {'Comedy', 'Romance', 'Drama'}

Top recommendations for User 2:
1. MovieF (Features: {'Comedy', 'Action', 'Crime'})
